In [1]:
import sys
sys.path.append('./KPConv-PyTorch/')

In [2]:
from utils.config import Config
from datasets.APRPointCloud import APRPointCloudDataset, APRPointCloudCollate, APRPointCloudCustomBatch, APRPointCloudSampler
from models.architectures import KPFCNN
from torch.utils.data import DataLoader
from utils.trainer import ModelTrainer
import numpy as np
import torch

In [3]:
class APRPointCloudConfig(Config):
    """
    Override the parameters you want to modify for this dataset
    """

    ####################
    # Dataset parameters
    ####################

    # Dataset name
    dataset = 'APRPointCloud'

    # Number of classes in the dataset (This value is overwritten by dataset class when Initializating dataset).
    num_classes = None

    # Type of task performed on this dataset (also overwritten)
    dataset_task = 'cloud_segmentation'

    # Number of CPU threads for the input pipeline
    input_threads = 0

    #########################
    # Architecture definition
    #########################

    # Define layers
    architecture = ['simple',
                    'resnetb',
                    'resnetb_strided',
                    'resnetb',
                    'resnetb',
                    'resnetb_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'resnetb_deformable_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'resnetb_deformable_strided',
                    'resnetb_deformable',
                    'resnetb_deformable',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary',
                    'nearest_upsample',
                    'unary']

    ###################
    # KPConv parameters
    ###################

    # Radius of the input sphere
    in_radius = 2.5

    # Number of kernel points
    num_kernel_points = 15

    # Size of the first subsampling grid in meter
    first_subsampling_dl = 0.1

    # Radius of convolution in "number grid cell". (2.5 is the standard value)
    conv_radius = 2.25
    
    # Radius of deformable convolution in "number grid cell". Larger so that deformed kernel can spread out
    deform_radius = 6.0

    # Radius of the area of influence of each kernel point in "number grid cell". (1.0 is the standard value)
    KP_extent = 1.5

    # Behavior of convolutions in ('constant', 'linear', 'gaussian')
    KP_influence = 'linear' #linear

    # Aggregation function of KPConv in ('closest', 'sum')
    aggregation_mode = 'sum'

    # Choice of input features
    first_features_dim = 128
    in_features_dim = 1

    # Can the network learn modulations
    modulated = False

    # Batch normalization parameters
    use_batch_norm = True
    batch_norm_momentum = 0.02

    # Deformable offset loss
    # 'point2point' fitting geometry by penalizing distance from deform point to input points
    # 'point2plane' fitting geometry by penalizing distance from deform point to input point triplet (not implemented)
    deform_fitting_mode = 'point2point'
    deform_fitting_power = 1.0              # Multiplier for the fitting/repulsive loss
    deform_lr_factor = 0.1                  # Multiplier for learning rate applied to the deformations
    repulse_extent = 1.2                    # Distance of repulsion for deformed kernel points

    #####################
    # Training parameters
    #####################

    # Maximal number of epochs
    max_epoch = 500

    # Learning rate management
    learning_rate = 1e-4
    momentum = 0.98
    lr_decays = {i: 0.1 ** (1 / 150) for i in range(1, max_epoch)}
    grad_clip_norm = 100.0

    # Number of batch
    batch_num = 10

    # Number of steps per epochs
    epoch_steps = 200

    # Number of validation examples per epoch
    validation_size = 20

    # Number of epoch between each checkpoint
    checkpoint_gap = 20

    # Augmentations
    augment_scale_anisotropic = True
    augment_symmetries = [True, False, False]
    augment_rotation = 'vertical'
    augment_scale_min = 0.8
    augment_scale_max = 1.2
    augment_noise = 0.001
    augment_color = 0.8

    # The way we balance segmentation loss
    #   > 'none': Each point in the whole batch has the same contribution.
    #   > 'class': Each class has the same contribution (points are weighted according to class balance)
    #   > 'batch': Each cloud in the batch has the same contribution (points are weighted according cloud sizes)
    segloss_balance = 'class'
    #class_w = [0.02, 0.98]

    # Do we need to save convergence
    saving = True
    saving_path = None

In [4]:
config = APRPointCloudConfig()
train_dataset = APRPointCloudDataset(config, set = 'training')
validation_dataset = APRPointCloudDataset(config, set = 'validation')

training_sampler = APRPointCloudSampler(train_dataset)
validation_sampler = APRPointCloudSampler(validation_dataset)


Preparing KDTree for cloud t000, subsampled at 0.100
Shape of the loaded point cloud = (225567, 5)
6.3 MB loaded in 2.6s

Preparing KDTree for cloud t001_2, subsampled at 0.100
Shape of the loaded point cloud = (164530, 5)
4.6 MB loaded in 1.9s

Preparing KDTree for cloud t002, subsampled at 0.100
Shape of the loaded point cloud = (214159, 5)
6.0 MB loaded in 2.3s

Preparing KDTree for cloud t006, subsampled at 0.100
Shape of the loaded point cloud = (214399, 5)
6.0 MB loaded in 2.4s

Preparing KDTree for cloud t008, subsampled at 0.100
Shape of the loaded point cloud = (217210, 5)
6.1 MB loaded in 2.4s

Preparing KDTree for cloud t008_2, subsampled at 0.100
Shape of the loaded point cloud = (143503, 5)
4.0 MB loaded in 1.5s

Preparing KDTree for cloud t011_2, subsampled at 0.100
Shape of the loaded point cloud = (154412, 5)
4.3 MB loaded in 1.6s

Preparing KDTree for cloud t012, subsampled at 0.100
Shape of the loaded point cloud = (215785, 5)
6.0 MB loaded in 2.8s

Preparing KDTree 

In [5]:
training_loader = DataLoader(train_dataset,
                            collate_fn=APRPointCloudCollate,
                            sampler=training_sampler,)

validation_loader = DataLoader(validation_dataset,
                              collate_fn=APRPointCloudCollate,
                              sampler=training_sampler,)

training_sampler.calibration(training_loader)
validation_sampler.calibration(validation_loader)

net = KPFCNN(config, train_dataset.label_values, train_dataset.ignored_labels)
trainer = ModelTrainer(net, config)


Starting Calibration (use verbose=True for more details)
Calibration done in 0.0s


Starting Calibration (use verbose=True for more details)
Calibration done in 0.0s



In [ ]:
trainer.train(net, training_loader, validation_loader, config)

e000-i0000 => L=14.619 acc= 25% / t(ms):  42.9 3122.4 307.1)
e000-i0002 => L=14.530 acc= 19% / t(ms):  34.1 248.3 276.4)
e000-i0004 => L=14.187 acc= 16% / t(ms):  33.8 245.5 276.2)
e000-i0006 => L=14.057 acc= 25% / t(ms):  33.4 242.2 277.9)
e000-i0008 => L=13.828 acc= 27% / t(ms):  33.1 243.3 277.4)
e000-i0010 => L=13.814 acc= 59% / t(ms):  32.9 241.1 277.9)
e000-i0012 => L=13.827 acc= 22% / t(ms):  33.4 239.4 278.8)
e000-i0014 => L=13.633 acc= 43% / t(ms):  35.2 239.1 278.4)
e000-i0016 => L=13.331 acc= 36% / t(ms):  35.4 240.9 279.7)
e000-i0018 => L=13.448 acc= 32% / t(ms):  35.8 239.9 279.8)
e000-i0020 => L=13.457 acc= 24% / t(ms):  36.4 240.4 279.8)
e000-i0022 => L=13.138 acc= 42% / t(ms):  36.5 240.6 280.2)
e000-i0024 => L=12.820 acc= 40% / t(ms):  35.9 239.8 281.6)
e000-i0026 => L=12.967 acc= 44% / t(ms):  35.6 240.9 283.6)
e000-i0028 => L=12.517 acc= 41% / t(ms):  35.7 240.8 284.4)
e000-i0030 => L=12.733 acc= 30% / t(ms):  35.3 240.4 283.5)
e000-i0032 => L=12.781 acc= 65% / t(ms)

e001-i0018 => L=10.040 acc= 69% / t(ms):  49.8 337.5 363.2)
e001-i0020 => L=9.961 acc= 74% / t(ms):  47.9 327.8 356.0)
e001-i0022 => L=9.853 acc= 82% / t(ms):  46.9 315.6 345.1)
e001-i0024 => L=9.876 acc= 89% / t(ms):  46.6 307.8 337.1)
e001-i0026 => L=10.389 acc= 78% / t(ms):  46.3 302.0 331.0)
e001-i0028 => L=10.217 acc= 87% / t(ms):  44.2 297.3 324.7)
e001-i0030 => L=9.700 acc= 81% / t(ms):  43.4 290.6 321.7)
e001-i0032 => L=9.954 acc= 79% / t(ms):  44.3 292.5 325.4)
e001-i0034 => L=10.004 acc= 87% / t(ms):  45.0 296.2 327.1)
e001-i0036 => L=9.929 acc= 84% / t(ms):  49.3 313.9 340.0)
e001-i0038 => L=9.705 acc= 85% / t(ms):  52.3 328.9 355.6)
e001-i0039 => L=9.936 acc= 78% / t(ms):  53.9 338.7 370.8)
e001-i0041 => L=9.718 acc= 87% / t(ms):  56.9 352.9 390.1)
e001-i0042 => L=9.609 acc= 85% / t(ms):  58.4 365.9 396.9)
e001-i0044 => L=9.818 acc= 75% / t(ms):  60.3 382.1 410.9)
e001-i0045 => L=9.658 acc= 84% / t(ms):  61.7 389.6 420.1)
e001-i0046 => L=9.842 acc= 75% / t(ms):  61.7 397.0 

e002-i0000 => L=8.880 acc= 90% / t(ms): 51014.0 308.7 335.1)
e002-i0002 => L=9.312 acc= 88% / t(ms):  43.2 308.6 322.6)
e002-i0004 => L=9.234 acc= 91% / t(ms):  44.9 309.1 326.2)
e002-i0006 => L=9.410 acc= 87% / t(ms):  44.8 307.2 332.0)
e002-i0008 => L=9.243 acc= 79% / t(ms):  47.5 315.2 342.2)
e002-i0010 => L=8.921 acc= 94% / t(ms):  55.9 321.2 356.6)
e002-i0012 => L=8.895 acc= 92% / t(ms):  58.5 328.3 364.2)
e002-i0014 => L=9.234 acc= 87% / t(ms):  57.4 331.7 371.2)
e002-i0016 => L=8.938 acc= 90% / t(ms):  58.5 342.2 378.6)
e002-i0018 => L=9.150 acc= 69% / t(ms):  59.2 355.1 390.7)
e002-i0020 => L=9.067 acc= 78% / t(ms):  60.2 363.4 400.0)
e002-i0022 => L=9.153 acc= 80% / t(ms):  61.3 373.1 405.1)
e002-i0024 => L=9.337 acc= 77% / t(ms):  62.4 377.2 413.9)
e002-i0026 => L=9.331 acc= 73% / t(ms):  65.3 382.2 416.6)
e002-i0028 => L=9.217 acc= 68% / t(ms):  65.0 388.0 419.0)
e002-i0030 => L=8.715 acc= 87% / t(ms):  64.1 389.4 422.4)
e002-i0032 => L=9.389 acc= 92% / t(ms):  64.3 392.8 42

Validation : 535.0% (timings : 173.91 194.50)
Validation : 555.0% (timings : 169.00 186.55)
Validation : 575.0% (timings : 168.21 180.74)
Validation : 595.0% (timings : 167.08 179.47)
Validation : 610.0% (timings : 168.12 177.93)
Validation : 630.0% (timings : 163.17 171.21)
Validation : 650.0% (timings : 158.78 164.79)
Validation : 670.0% (timings : 154.09 159.88)
Validation : 690.0% (timings : 150.88 157.94)
Validation : 710.0% (timings : 147.38 152.90)
Validation : 730.0% (timings : 143.26 148.87)
Validation : 750.0% (timings : 143.16 146.30)
Validation : 770.0% (timings : 148.55 149.20)
Validation : 785.0% (timings : 152.37 152.48)
Validation : 800.0% (timings : 160.99 164.97)
Validation : 820.0% (timings : 149.51 173.03)
Validation : 840.0% (timings : 142.80 177.96)
Validation : 860.0% (timings : 136.17 181.93)
Validation : 880.0% (timings : 127.14 184.27)
Validation : 900.0% (timings : 119.71 185.98)
Validation : 920.0% (timings : 117.42 190.26)
Validation : 940.0% (timings : 114

Validation : 295.0% (timings : 84.14 181.17)
Validation : 315.0% (timings : 85.12 182.19)
Validation : 335.0% (timings : 87.41 181.38)
Validation : 355.0% (timings : 89.26 181.35)
Validation : 375.0% (timings : 89.01 180.92)
Validation : 395.0% (timings : 87.49 179.18)
Validation : 420.0% (timings : 84.54 170.30)
Validation : 450.0% (timings : 78.11 159.91)
Validation : 480.0% (timings : 73.78 152.48)
Validation : 505.0% (timings : 70.50 149.60)
Validation : 530.0% (timings : 69.63 147.93)
Validation : 555.0% (timings : 70.95 150.30)
Validation : 575.0% (timings : 73.98 155.95)
Validation : 595.0% (timings : 76.92 160.01)
Validation : 615.0% (timings : 80.79 163.17)
Validation : 635.0% (timings : 81.28 167.76)
Validation : 655.0% (timings : 83.53 169.51)
Validation : 675.0% (timings : 84.61 171.21)
Validation : 695.0% (timings : 87.74 175.75)
Validation : 715.0% (timings : 90.86 180.57)
Validation : 735.0% (timings : 92.41 183.03)
Validation : 755.0% (timings : 93.08 183.64)
Validation

...

e031-i0150 => L=6.601 acc= 77% / t(ms):  58.1 358.2 395.9)
e031-i0152 => L=6.933 acc= 96% / t(ms):  57.4 365.2 401.9)
e031-i0154 => L=6.916 acc= 90% / t(ms):  59.7 366.6 402.6)
e031-i0156 => L=7.618 acc= 93% / t(ms):  58.5 365.3 403.1)
e031-i0158 => L=6.658 acc= 79% / t(ms):  56.0 353.2 388.4)
e031-i0160 => L=6.723 acc= 96% / t(ms):  61.8 345.6 389.0)
e031-i0162 => L=7.038 acc= 86% / t(ms):  60.7 346.9 386.0)
e031-i0164 => L=7.136 acc= 91% / t(ms):  57.4 337.8 376.7)
e031-i0166 => L=7.169 acc= 81% / t(ms):  54.7 326.7 365.2)
e031-i0168 => L=6.811 acc= 85% / t(ms):  53.2 316.2 356.7)
e031-i0170 => L=6.495 acc= 93% / t(ms):  51.5 308.6 353.6)
e031-i0172 => L=7.467 acc= 86% / t(ms):  49.8 306.0 351.4)
e031-i0174 => L=6.518 acc= 97% / t(ms):  48.9 316.3 357.9)
e031-i0176 => L=7.302 acc= 88% / t(ms):  49.5 321.7 368.1)
e031-i0178 => L=6.880 acc= 83% / t(ms):  50.1 328.0 375.8)
e031-i0180 => L=6.636 acc= 88% / t(ms):  50.5 333.3 378.0)
e031-i0182 => L=6.764 acc= 78% / t(ms):  50.5 338.5 382.

e032-i0163 => L=7.329 acc= 89% / t(ms):  66.0 377.3 427.3)
e032-i0165 => L=7.022 acc= 97% / t(ms):  64.0 372.2 417.9)
e032-i0167 => L=7.154 acc= 84% / t(ms):  61.5 360.5 401.4)
e032-i0169 => L=7.328 acc= 76% / t(ms):  57.8 351.4 389.2)
e032-i0171 => L=7.112 acc= 95% / t(ms):  55.8 340.3 380.4)
e032-i0173 => L=6.859 acc= 87% / t(ms):  56.1 345.0 382.3)
e032-i0175 => L=6.823 acc= 86% / t(ms):  56.8 346.2 383.2)
e032-i0177 => L=6.956 acc= 90% / t(ms):  57.6 351.5 385.6)
e032-i0179 => L=7.240 acc= 78% / t(ms):  57.2 350.0 390.9)
e032-i0181 => L=7.973 acc= 97% / t(ms):  59.5 354.6 392.0)
e032-i0183 => L=7.338 acc= 76% / t(ms):  59.9 357.6 397.7)
e032-i0185 => L=6.983 acc= 78% / t(ms):  59.1 358.2 399.2)
e032-i0187 => L=7.508 acc= 88% / t(ms):  61.2 360.7 400.9)
e032-i0189 => L=6.507 acc= 94% / t(ms):  59.1 358.9 402.7)
e032-i0191 => L=6.967 acc= 95% / t(ms):  59.2 360.7 402.0)
e032-i0193 => L=7.822 acc= 84% / t(ms):  59.3 357.4 403.3)
e032-i0195 => L=7.882 acc= 87% / t(ms):  59.4 359.7 403.

e033-i0176 => L=7.853 acc= 96% / t(ms):  59.1 353.8 396.9)
e033-i0178 => L=7.414 acc= 67% / t(ms):  59.9 354.0 398.7)
e033-i0180 => L=7.075 acc= 89% / t(ms):  58.4 353.4 401.0)
e033-i0182 => L=6.605 acc= 86% / t(ms):  58.4 356.6 397.6)
e033-i0184 => L=7.228 acc= 90% / t(ms):  57.7 357.6 399.9)
e033-i0186 => L=6.881 acc= 86% / t(ms):  56.5 353.3 391.2)
e033-i0188 => L=7.165 acc= 84% / t(ms):  53.1 340.6 379.2)
e033-i0190 => L=7.306 acc= 98% / t(ms):  52.4 331.5 368.5)
e033-i0192 => L=7.067 acc= 91% / t(ms):  51.1 320.0 357.0)
e033-i0194 => L=7.368 acc= 93% / t(ms):  50.7 316.2 353.6)
e033-i0196 => L=7.113 acc= 97% / t(ms):  49.8 316.0 356.1)
e033-i0198 => L=7.587 acc= 94% / t(ms):  51.4 325.2 364.3)
Validation : 20.0% (timings : 16.16 34.80)
Validation : 45.0% (timings : 27.88 62.97)
Validation : 70.0% (timings : 42.46 82.13)
Validation : 95.0% (timings : 49.87 100.47)
Validation : 120.0% (timings : 56.00 112.55)
Validation : 145.0% (timings : 57.48 120.09)
Validation : 170.0% (timings 

e034-i0187 => L=6.891 acc= 93% / t(ms):  65.9 378.4 421.5)
e034-i0189 => L=7.110 acc= 90% / t(ms):  68.2 382.7 425.7)
e034-i0191 => L=7.583 acc= 97% / t(ms):  67.8 385.7 420.5)
e034-i0193 => L=7.291 acc= 89% / t(ms):  68.2 379.8 423.2)
e034-i0195 => L=6.901 acc= 80% / t(ms):  69.1 378.5 416.4)
e034-i0197 => L=7.043 acc= 79% / t(ms):  67.5 373.3 413.6)
e034-i0199 => L=6.546 acc= 85% / t(ms):  64.9 364.8 405.6)
Validation : 25.0% (timings : 13.17 32.62)
Validation : 60.0% (timings : 23.62 57.82)
Validation : 90.0% (timings : 31.03 75.44)
Validation : 120.0% (timings : 39.06 91.78)
Validation : 145.0% (timings : 44.15 103.06)
Validation : 170.0% (timings : 50.30 116.06)
Validation : 195.0% (timings : 55.34 124.63)
Validation : 220.0% (timings : 59.30 132.93)
Validation : 245.0% (timings : 62.59 140.65)
Validation : 270.0% (timings : 65.13 143.43)
Validation : 295.0% (timings : 66.29 147.83)
Validation : 320.0% (timings : 68.20 149.39)
Validation : 345.0% (timings : 68.16 148.47)
Validatio

e035-i0199 => L=6.896 acc= 84% / t(ms):  52.9 317.8 352.1)
Validation : 25.0% (timings : 16.81 36.04)
Validation : 50.0% (timings : 32.16 65.26)
Validation : 75.0% (timings : 41.39 86.19)
Validation : 95.0% (timings : 48.51 102.86)
Validation : 120.0% (timings : 54.19 113.54)
Validation : 145.0% (timings : 58.83 122.04)
Validation : 170.0% (timings : 61.23 128.41)
Validation : 195.0% (timings : 63.98 130.99)
Validation : 220.0% (timings : 68.90 142.25)
Validation : 240.0% (timings : 91.71 146.54)
